# Workshop: Olympic Medals

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/Olympic_rings_without_rims.svg/1920px-Olympic_rings_without_rims.png" width=300>

First clone the repository at [https://github.com/daniprec/BAM-Coding-Lab](https://github.com/daniprec/BAM-Coding-Lab)

You will find the data inside `data`.


---

In [1]:
import pandas as pd

## 🔍 Data Loading and EDA

We will analyze the "olympic_medals.csv" file, which includes data on all Olympic athletes from 1908 to 2022, covering both the Summer and Winter Games.

In [3]:
# ".." means "go up one level" or "parent directory"
path_csv = "../data/olympics_medals.csv"

# We use "read_csv" to load a "csv" file
df = pd.read_csv(path_csv)

# "head(n)" returns the first n rows
df.head(5)

,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
0,1908 Summer Olympics,5,ANZ,Athletics,"100 metres, Men",56265,Ernest Hutcheon,64710,DNS,NaN,False
1,1908 Summer Olympics,5,ANZ,Athletics,"400 metres, Men",56313,Henry Murray,64756,DNS,NaN,False
2,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Harvey Sutton,64808,3 h8 r1/2,NaN,False
3,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Guy Haskins,922519,DNS,NaN,False
4,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Joseph Lynch,64735,DNS,NaN,False


Take a moment to review the column names. Do we understand their meanings?

In [5]:
# "columns" returns the column names
print(df.columns)

Index(['edition', 'edition_id', 'country_noc', 'sport', 'event', 'result_id',
       'athlete', 'athlete_id', 'pos', 'medal', 'isTeamSport'],
      dtype='object')


In [4]:
# "shape" returns the number of rows and columns
print(df.shape)

(316834, 11)


In [6]:
# "info()" prints a concise summary of a dataframe
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316834 entries, 0 to 316833
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   edition      316834 non-null  object
 1   edition_id   316834 non-null  int64 
 2   country_noc  316834 non-null  object
 3   sport        316834 non-null  object
 4   event        316834 non-null  object
 5   result_id    316834 non-null  int64 
 6   athlete      316834 non-null  object
 7   athlete_id   316834 non-null  int64 
 8   pos          316834 non-null  object
 9   medal        44687 non-null   object
 10  isTeamSport  316834 non-null  bool  
dtypes: bool(1), int64(3), object(7)
memory usage: 24.5+ MB
None


---

## ⚡ Data Cleaning and Processing

Before we can analyze our dataset, it's important to clean and process the data.

Clean data ensures that our insights are accurate and meaningful.

### NaN Values

During our Exploratory Data Analysis (EDA), we noticed that the **"medal"** column contains NaN values.

When this happens, we first must ask us what does the missing value mean.

These NaNs represent the absence of a medal. To avoid confusion, let's replace these missing values with something more meaningful, such as `"No Medal"`, to clarify that the athlete did not win a medal.

In [9]:
# We first see the values contained inside the "medal" column
print(df["medal"].value_counts())

medal
Gold      15072
Bronze    14939
Silver    14676
Name: count, dtype: int64


In [10]:
# Replace NaNs with "No"
df["medal"].fillna("No", inplace=True)

print(df["medal"].value_counts())

medal
No        272147
Gold       15072
Bronze     14939
Silver     14676
Name: count, dtype: int64


### Splitting a Column

The **"edition"** column contains both the season and year, for example, "2020 Summer Olympics". Let's split this into two separate columns:

- **"year"**, for example, "2020" (this should be an integer).
- **"season"**, for example, "Summer" (as a string).

In [7]:
print(df["edition"].value_counts())

edition
2020 Summer Olympics        14816
2000 Summer Olympics        14041
1996 Summer Olympics        14018
2008 Summer Olympics        13980
2016 Summer Olympics        13942
2004 Summer Olympics        13762
1992 Summer Olympics        13516
2012 Summer Olympics        13196
1988 Summer Olympics        12780
1972 Summer Olympics        10903
1984 Summer Olympics        10113
1968 Summer Olympics         9377
1952 Summer Olympics         9301
1976 Summer Olympics         8990
1960 Summer Olympics         8573
1964 Summer Olympics         8208
1980 Summer Olympics         7499
1936 Summer Olympics         7397
1948 Summer Olympics         7162
1924 Summer Olympics         6503
1928 Summer Olympics         5780
1912 Summer Olympics         5427
2022 Winter Olympics         5408
1956 Summer Olympics         5317
2018 Winter Olympics         5196
2014 Winter Olympics         4984
1920 Summer Olympics         4676
2010 Winter Olympics         4467
2006 Winter Olympics         4446
1900 S

In [8]:
# We use the string method "split"
# Alternative: use regex with "re" library (more robust)
df["year"] = df["edition"].apply(lambda x: int(x.split()[0]))
print(df["year"].value_counts()[:5])

df["season"] = df["edition"].apply(lambda x: x.split()[1])
print(df["season"].value_counts())

year
1992    17229
1988    15946
2020    14816
2000    14041
1996    14018
Name: count, dtype: int64
season
Summer          254658
Winter           61853
Equestrian         300
Intercalated        23
Name: count, dtype: int64


In the **"event"** column, you'll notice that some entries include the tags "Men" or "Women" to indicate gender. Let's split this column into two new columns:

- **"event_simple"**: This will contain the event name without gender.
- **"gender"**: This will be either "Men", "Women", or "Unknown" if no gender is specified.

For example, the value `"Football, Men"` will be split into `"Football"` and `"Men"`.

In [11]:
def split_event_gender(event: str) -> tuple:
  if "women" in event.lower():
    return event.split(", ")[0], "Women"
  elif "men" in event.lower():
    return event.split(", ")[0], "Men"
  else:
    return event, "Unknown"

df[["event_simple", "gender"]] = df["event"].apply(split_event_gender).apply(pd.Series)

---

## 🛠️ Building a Pipeline Script

After loading and cleaning the data, it’s a good idea to organize everything into a function. This makes the process reusable and keeps the code tidy.

In the same folder as this notebook, create a file called `olympics.py`. Inside, define a function `load_and_preprocess()` that:

- Takes the path to the `olympics_medals.csv` file as input.
- Loads the data, cleans it, and adds new columns like `year` and `gender`.
- Returns the cleaned dataframe.

This way, you can easily regenerate the processed data from raw files, keeping everything streamlined and efficient!